In [ ]:
from __future__ import unicode_literals
from __future__ import division
import numpy as np
import math
import sys
from fractions import Fraction
import warnings
from collections import Counter
from nltk import ngrams

from builtins import zip
from builtins import range
from past.utils import old_div
from builtins import object
from collections import defaultdict
import math
import re
import json

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tyfann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from itertools import chain, product


def _generate_enums(hypothesis, reference, preprocess=str.lower):
    """
    Takes in string inputs for hypothesis and reference and returns
    enumerated word lists for each of them

    :param hypothesis: hypothesis string
    :type hypothesis: str
    :param reference: reference string
    :type reference: str
    :preprocess: preprocessing method (default str.lower)
    :type preprocess: method
    :return: enumerated words list
    :rtype: list of 2D tuples, list of 2D tuples
    """
    hypothesis_list = list(enumerate(preprocess(hypothesis).split()))
    reference_list = list(enumerate(preprocess(reference).split()))
    return hypothesis_list, reference_list


def exact_match(hypothesis, reference):
    """
    matches exact words in hypothesis and reference
    and returns a word mapping based on the enumerated
    word id between hypothesis and reference

    :param hypothesis: hypothesis string
    :type hypothesis: str
    :param reference: reference string
    :type reference: str
    :return: enumerated matched tuples, enumerated unmatched hypothesis tuples,
             enumerated unmatched reference tuples
    :rtype: list of 2D tuples, list of 2D tuples,  list of 2D tuples
    """
    hypothesis_list, reference_list = _generate_enums(hypothesis, reference)
    return _match_enums(hypothesis_list, reference_list)



def _match_enums(enum_hypothesis_list, enum_reference_list):
    """
    matches exact words in hypothesis and reference and returns
    a word mapping between enum_hypothesis_list and enum_reference_list
    based on the enumerated word id.

    :param enum_hypothesis_list: enumerated hypothesis list
    :type enum_hypothesis_list: list of tuples
    :param enum_reference_list: enumerated reference list
    :type enum_reference_list: list of 2D tuples
    :return: enumerated matched tuples, enumerated unmatched hypothesis tuples,
             enumerated unmatched reference tuples
    :rtype: list of 2D tuples, list of 2D tuples,  list of 2D tuples
    """
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        for j in range(len(enum_reference_list))[::-1]:
            if enum_hypothesis_list[i][1] == enum_reference_list[j][1]:
                word_match.append(
                    (enum_hypothesis_list[i][0], enum_reference_list[j][0])
                )
                (enum_hypothesis_list.pop(i)[1], enum_reference_list.pop(j)[1])
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _enum_stem_match(
    enum_hypothesis_list, enum_reference_list, stemmer=PorterStemmer()
):
    """
    Stems each word and matches them in hypothesis and reference
    and returns a word mapping between enum_hypothesis_list and
    enum_reference_list based on the enumerated word id. The function also
    returns a enumerated list of unmatched words for hypothesis and reference.

    :param enum_hypothesis_list:
    :type enum_hypothesis_list:
    :param enum_reference_list:
    :type enum_reference_list:
    :param stemmer: nltk.stem.api.StemmerI object (default PorterStemmer())
    :type stemmer: nltk.stem.api.StemmerI or any class that implements a stem method
    :return: enumerated matched tuples, enumerated unmatched hypothesis tuples,
             enumerated unmatched reference tuples
    :rtype: list of 2D tuples, list of 2D tuples,  list of 2D tuples
    """
    stemmed_enum_list1 = [
        (word_pair[0], stemmer.stem(word_pair[1])) for word_pair in enum_hypothesis_list
    ]

    stemmed_enum_list2 = [
        (word_pair[0], stemmer.stem(word_pair[1])) for word_pair in enum_reference_list
    ]

    word_match, enum_unmat_hypo_list, enum_unmat_ref_list = _match_enums(
        stemmed_enum_list1, stemmed_enum_list2
    )

    enum_unmat_hypo_list = (
        list(zip(*enum_unmat_hypo_list)) if len(enum_unmat_hypo_list) > 0 else []
    )

    enum_unmat_ref_list = (
        list(zip(*enum_unmat_ref_list)) if len(enum_unmat_ref_list) > 0 else []
    )

    enum_hypothesis_list = list(
        filter(lambda x: x[0] not in enum_unmat_hypo_list, enum_hypothesis_list)
    )

    enum_reference_list = list(
        filter(lambda x: x[0] not in enum_unmat_ref_list, enum_reference_list)
    )

    return word_match, enum_hypothesis_list, enum_reference_list


def stem_match(hypothesis, reference, stemmer=PorterStemmer()):
    """
    Stems each word and matches them in hypothesis and reference
    and returns a word mapping between hypothesis and reference

    :param hypothesis:
    :type hypothesis:
    :param reference:
    :type reference:
    :param stemmer: nltk.stem.api.StemmerI object (default PorterStemmer())
    :type stemmer: nltk.stem.api.StemmerI or any class that
                   implements a stem method
    :return: enumerated matched tuples, enumerated unmatched hypothesis tuples,
             enumerated unmatched reference tuples
    :rtype: list of 2D tuples, list of 2D tuples,  list of 2D tuples
    """
    enum_hypothesis_list, enum_reference_list = _generate_enums(hypothesis, reference)
    return _enum_stem_match(enum_hypothesis_list, enum_reference_list, stemmer=stemmer)



def _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, wordnet=wordnet):
    """
    Matches each word in reference to a word in hypothesis
    if any synonym of a hypothesis word is the exact match
    to the reference word.

    :param enum_hypothesis_list: enumerated hypothesis list
    :param enum_reference_list: enumerated reference list
    :param wordnet: a wordnet corpus reader object (default nltk.corpus.wordnet)
    :type wordnet: WordNetCorpusReader
    :return: list of matched tuples, unmatched hypothesis list, unmatched reference list
    :rtype:  list of tuples, list of tuples, list of tuples

    """
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        hypothesis_syns = set(
            chain.from_iterable(
                (
                    lemma.name()
                    for lemma in synset.lemmas()
                    if lemma.name().find("_") < 0
                )
                for synset in wordnet.synsets(enum_hypothesis_list[i][1])
            )
        ).union({enum_hypothesis_list[i][1]})
        for j in range(len(enum_reference_list))[::-1]:
            if enum_reference_list[j][1] in hypothesis_syns:
                word_match.append(
                    (enum_hypothesis_list[i][0], enum_reference_list[j][0])
                )
                enum_hypothesis_list.pop(i), enum_reference_list.pop(j)
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def wordnetsyn_match(hypothesis, reference, wordnet=wordnet):
    """
    Matches each word in reference to a word in hypothesis if any synonym
    of a hypothesis word is the exact match to the reference word.

    :param hypothesis: hypothesis string
    :param reference: reference string
    :param wordnet: a wordnet corpus reader object (default nltk.corpus.wordnet)
    :type wordnet: WordNetCorpusReader
    :return: list of mapped tuples
    :rtype: list of tuples
    """
    enum_hypothesis_list, enum_reference_list = _generate_enums(hypothesis, reference)
    return _enum_wordnetsyn_match(
        enum_hypothesis_list, enum_reference_list, wordnet=wordnet
    )



def _enum_allign_words(
    enum_hypothesis_list, enum_reference_list, stemmer=PorterStemmer(), wordnet=wordnet
):
    """
    Aligns/matches words in the hypothesis to reference by sequentially
    applying exact match, stemmed match and wordnet based synonym match.
    in case there are multiple matches the match which has the least number
    of crossing is chosen. Takes enumerated list as input instead of
    string input

    :param enum_hypothesis_list: enumerated hypothesis list
    :param enum_reference_list: enumerated reference list
    :param stemmer: nltk.stem.api.StemmerI object (default PorterStemmer())
    :type stemmer: nltk.stem.api.StemmerI or any class that implements a stem method
    :param wordnet: a wordnet corpus reader object (default nltk.corpus.wordnet)
    :type wordnet: WordNetCorpusReader
    :return: sorted list of matched tuples, unmatched hypothesis list,
             unmatched reference list
    :rtype: list of tuples, list of tuples, list of tuples
    """
    exact_matches, enum_hypothesis_list, enum_reference_list = _match_enums(
        enum_hypothesis_list, enum_reference_list
    )

    stem_matches, enum_hypothesis_list, enum_reference_list = _enum_stem_match(
        enum_hypothesis_list, enum_reference_list, stemmer=stemmer
    )

    wns_matches, enum_hypothesis_list, enum_reference_list = _enum_wordnetsyn_match(
        enum_hypothesis_list, enum_reference_list, wordnet=wordnet
    )

    return (
        sorted(
            exact_matches + stem_matches + wns_matches, key=lambda wordpair: wordpair[0]
        ),
        enum_hypothesis_list,
        enum_reference_list,
    )


def allign_words(hypothesis, reference, stemmer=PorterStemmer(), wordnet=wordnet):
    """
    Aligns/matches words in the hypothesis to reference by sequentially
    applying exact match, stemmed match and wordnet based synonym match.
    In case there are multiple matches the match which has the least number
    of crossing is chosen.

    :param hypothesis: hypothesis string
    :param reference: reference string
    :param stemmer: nltk.stem.api.StemmerI object (default PorterStemmer())
    :type stemmer: nltk.stem.api.StemmerI or any class that implements a stem method
    :param wordnet: a wordnet corpus reader object (default nltk.corpus.wordnet)
    :type wordnet: WordNetCorpusReader
    :return: sorted list of matched tuples, unmatched hypothesis list, unmatched reference list
    :rtype: list of tuples, list of tuples, list of tuples
    """
    enum_hypothesis_list, enum_reference_list = _generate_enums(hypothesis, reference)
    return _enum_allign_words(
        enum_hypothesis_list, enum_reference_list, stemmer=stemmer, wordnet=wordnet
    )



def _count_chunks(matches):
    """
    Counts the fewest possible number of chunks such that matched unigrams
    of each chunk are adjacent to each other. This is used to caluclate the
    fragmentation part of the metric.

    :param matches: list containing a mapping of matched words (output of allign_words)
    :return: Number of chunks a sentence is divided into post allignment
    :rtype: int
    """
    i = 0
    chunks = 1
    while i < len(matches) - 1:
        if (matches[i + 1][0] == matches[i][0] + 1) and (
            matches[i + 1][1] == matches[i][1] + 1
        ):
            i += 1
            continue
        i += 1
        chunks += 1
    return chunks


def single_meteor_score(
    reference,
    hypothesis,
    preprocess=str.lower,
    stemmer=PorterStemmer(),
    wordnet=wordnet,
    alpha=0.85,
    beta=2.35,
    gamma=0.45,
    w_1=1,
    w_2=0.8,
    w_3=0.6
):
    """
    Calculates METEOR score for single hypothesis and reference as per
    "Meteor: An Automatic Metric for MT Evaluation with HighLevels of
    Correlation with Human Judgments" by Alon Lavie and Abhaya Agarwal,
    in Proceedings of ACL.
    http://www.cs.cmu.edu/~alavie/METEOR/pdf/Lavie-Agarwal-2007-METEOR.pdf


    >>> hypothesis1 = 'It is a guide to action which ensures that the military always obeys the commands of the party'

    >>> reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'


    >>> round(single_meteor_score(reference1, hypothesis1),4)
    0.7398

        If there is no words match during the alignment the method returns the
        score as 0. We can safely  return a zero instead of raising a
        division by zero error as no match usually implies a bad translation.

    >>> round(meteor_score('this is a cat', 'non matching hypothesis'),4)
    0.0

    :param references: reference sentences
    :type references: list(str)
    :param hypothesis: a hypothesis sentence
    :type hypothesis: str
    :param preprocess: preprocessing function (default str.lower)
    :type preprocess: method
    :param stemmer: nltk.stem.api.StemmerI object (default PorterStemmer())
    :type stemmer: nltk.stem.api.StemmerI or any class that implements a stem method
    :param wordnet: a wordnet corpus reader object (default nltk.corpus.wordnet)
    :type wordnet: WordNetCorpusReader
    :param alpha: parameter for controlling relative weights of precision and recall.
    :type alpha: float
    :param beta: parameter for controlling shape of penalty as a
                 function of as a function of fragmentation.
    :type beta: float
    :param gamma: relative weight assigned to fragmentation penality.
    :type gamma: float
    :return: The sentence-level METEOR score.
    :rtype: float
    """
    enum_hypothesis, enum_reference = _generate_enums(
        hypothesis, reference, preprocess=preprocess
    )
    translation_length = len(enum_hypothesis)
    #print(translation_length)
    reference_length = len(enum_reference)
    matches, _, _ = _enum_allign_words(enum_hypothesis, enum_reference, stemmer=stemmer)
    #print(matches)
    exact_m,_,_ = exact_match(hypothesis, reference)
    #print(exact_m)
    stem_m,_,_ = _enum_stem_match(enum_hypothesis, enum_reference, stemmer=PorterStemmer())
    #print(stem_m)
    syn_m,_,_ = _enum_wordnetsyn_match(enum_hypothesis, enum_reference, wordnet=wordnet)
    #print(syn_m)



    exact_count = len(exact_m)
    stem_matches = set(stem_m).difference(exact_m)
    stem_count = len((set(stem_m).difference(exact_m)))
    syn_matches = set(syn_m).difference(stem_m)
    syn_count = len((set(syn_m).difference(stem_m)))
    tot_matches = list(exact_m) + list(stem_matches) + list(syn_matches)
    tot_alligned_matches = sorted(
            tot_matches, key=lambda wordpair: wordpair[0]
        )
    tot_alligned_count = len(tot_alligned_matches)
    #print(tot_alligned_matches)
    #print(exact_count)
    #print(stem_count)
    #print(syn_count)
    #print(tot_alligned_count)
    matches_count = len(matches)
    #print(matches_count)
    try:
        precision = float(w_1*exact_count + w_2*stem_count + w_3*syn_count) / translation_length
        recall = float(w_1*exact_count + w_2*stem_count + w_3*syn_count) / reference_length
        #precision = float(matches_count) / translation_length
        #recall = float(matches_count) / reference_length
        fmean = (precision * recall) / (alpha * precision + (1 - alpha) * recall)
        chunk_count = float(_count_chunks(matches))
        frag_frac = chunk_count / matches_count
    except ZeroDivisionError:
        return 0.0
    if tot_alligned_count == translation_length and tot_alligned_count == reference_length:
        penalty = 0
    else:
        penalty = gamma * frag_frac ** beta
    return (1 - penalty) * fmean



def meteorn_score(
    references,
    hypothesis,
    preprocess=str.lower,
    stemmer=PorterStemmer(),
    wordnet=wordnet,
    alpha=0.85,
    beta=2.35,
    gamma=0.45,
    w_1=1,
    w_2=0.8,
    w_3=0.6
):
    """
    Calculates METEOR score for hypothesis with multiple references as
    described in "Meteor: An Automatic Metric for MT Evaluation with
    HighLevels of Correlation with Human Judgments" by Alon Lavie and
    Abhaya Agarwal, in Proceedings of ACL.
    http://www.cs.cmu.edu/~alavie/METEOR/pdf/Lavie-Agarwal-2007-METEOR.pdf


    In case of multiple references the best score is chosen. This method
    iterates over single_meteor_score and picks the best pair among all
    the references for a given hypothesis

    >>> hypothesis1 = 'It is a guide to action which ensures that the military always obeys the commands of the party'
    >>> hypothesis2 = 'It is to insure the troops forever hearing the activity guidebook that party direct'

    >>> reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'
    >>> reference2 = 'It is the guiding principle which guarantees the military forces always being under the command of the Party'
    >>> reference3 = 'It is the practical guide for the army always to heed the directions of the party'

    >>> round(meteor_score([reference1, reference2, reference3], hypothesis1),4)
    0.7398

        If there is no words match during the alignment the method returns the
        score as 0. We can safely  return a zero instead of raising a
        division by zero error as no match usually implies a bad translation.

    >>> round(meteor_score(['this is a cat'], 'non matching hypothesis'),4)
    0.0

    :param references: reference sentences
    :type references: list(str)
    :param hypothesis: a hypothesis sentence
    :type hypothesis: str
    :param preprocess: preprocessing function (default str.lower)
    :type preprocess: method
    :param stemmer: nltk.stem.api.StemmerI object (default PorterStemmer())
    :type stemmer: nltk.stem.api.StemmerI or any class that implements a stem method
    :param wordnet: a wordnet corpus reader object (default nltk.corpus.wordnet)
    :type wordnet: WordNetCorpusReader
    :param alpha: parameter for controlling relative weights of precision and recall.
    :type alpha: float
    :param beta: parameter for controlling shape of penalty as a function
                 of as a function of fragmentation.
    :type beta: float
    :param gamma: relative weight assigned to fragmentation penality.
    :type gamma: float
    :return: The sentence-level METEOR score.
    :rtype: float
    """
    return max(
        [
            single_meteor_score(
                reference,
                hypothesis,
                preprocess=preprocess,
                stemmer=stemmer,
                wordnet=wordnet,
                alpha=alpha,
                beta=beta,
                gamma=gamma,
            )
            for reference in references
        ]
    )

In [4]:
def calculate_log_MNEXT(file_path, ref_name, pred_name):

  refs=[]
  preds=[]
  with open(file_path, 'r', encoding='UTF-8') as f:
      org_data = json.load(f)
  # org_data = org_data[:100]

  for data in org_data:
      refs.append(data[ref_name])
      preds.append(data[pred_name])

  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  #refs_new = []
  for i in range(len(refs)):
      for ele in refs[i]:
          if ele in punc:
              refs[i] = refs[i].replace(ele, " ")
              refs[i] = re.sub(r'\s+', ' ', refs[i]).strip()

  #preds_new = []
  for i in range(len(refs)):
      if preds[i]==None:
          print(i)
      for ele in preds[i]:
          if ele in punc:
              preds[i] = preds[i].replace(ele, " ")
              preds[i] = re.sub(r'\s+', ' ', preds[i]).strip()

  log_MNEXT=[]

  for i in range(len(refs)):
      log_MNEXT.append(round(meteorn_score([refs[i]],preds[i]),2))
  log_MNEXT_score = (sum(log_MNEXT)/len(log_MNEXT))*100
  print(f'{file_path}/{pred_name}: {log_MNEXT_score}')

In [ ]:
calculate_log_MNEXT('../data/final_preprocessed_data/js_baseline/js_baseline_zeroshot_300.json', 'chatgpt')

In [12]:
calculate_log_MNEXT(f'../data/final_preprocessed_data/js_baseline/js_baseline_test_data_300_golden.json', 'msg', 'sim_msg')
# BM25 prediction
calculate_log_MNEXT(f'../data/final_preprocessed_data/js_baseline/js_baseline_test_data_300_bm25.json', 'msg', 'sim_msg')
# ChromaDB prediction
calculate_log_MNEXT(f'../data/final_preprocessed_data/js_baseline/js_baseline_rag_300_prompt.json', 'org_msg', 'sim_msg')
# Weaviate prediction
calculate_log_MNEXT(f'../data/final_preprocessed_data/js_baseline/js_baseline_weaviate_rag_300_prompt.json', 'org_msg', 'sim_msg')

../data/final_preprocessed_data/js_baseline/js_baseline_test_data_300_golden.json/sim_msg: 38.11599999999998
../data/final_preprocessed_data/js_baseline/js_baseline_test_data_300_bm25.json/sim_msg: 3.542999999999999
../data/final_preprocessed_data/js_baseline/js_baseline_rag_300_prompt.json/sim_msg: 3.4759999999999978
../data/final_preprocessed_data/js_baseline/js_baseline_weaviate_rag_300_prompt.json/sim_msg: 3.2719999999999985


In [16]:
calculate_log_MNEXT(f'../data/final_preprocessed_data/js_baseline/js_baseline_weaviate_rag_300_old.json', 'msg', 'chatgpt_rag')

../data/final_preprocessed_data/js_baseline/js_baseline_weaviate_rag_300_old.json/chatgpt_rag: 17.06


In [12]:
calculate_log_MNEXT('../data/final_preprocessed_data/discriminator/dis_train_data_retrieve.json', 'org_msg', 'sim_msg')
calculate_log_MNEXT('../data/final_preprocessed_data/discriminator/dis_train_data_retrieve_openai_embedding.json', 'org_msg', 'sim_msg')

../data/final_preprocessed_data/discriminator/dis_train_data_retrieve.json/sim_msg: 4.925361831033341
../data/final_preprocessed_data/discriminator/dis_train_data_retrieve_openai_embedding.json/sim_msg: 5.420397172669198


In [5]:
def calculate_log_MNEXT_by_pair(pred_data, ref_data):
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for ele in ref_data:
    if ele in punc:
      ref_data = ref_data.replace(ele, "")
  
  for ele in pred_data:
    if ele in punc:
      pred_data = pred_data.replace(ele, "")
  
  return round(meteorn_score([ref_data],pred_data),4)

In [10]:
from tqdm import tqdm
with open('../data/final_preprocessed_data/js_baseline/js_baseline_rag_300.json', 'r', encoding='UTF-8') as f:
  data = json.load(f)

highest_score = 0
highest_item = None
highest_index = -1

lowest_score = 1
lowest_item = None
lowest_index = -1
for index, item in tqdm(enumerate(data)):
    score = calculate_log_MNEXT_by_pair(item['chatgpt_rag'], item['msg'])
    if score > highest_score:
        highest_score = score
        highest_item = item
        highest_index = index
    if score < lowest_score:
        lowest_score = score
        lowest_item = item
        lowest_index = index

1000it [00:00, 1721.66it/s]


In [13]:
with open('../data/final_preprocessed_data/js_baseline/js_baseline_zeroshot_300.json', 'r', encoding='UTF-8') as f:
  zeroshot_data = json.load(f)

In [14]:
zeroshot_data[highest_index]

{'msg': 'update vertexComponentCount in WebGLPipeline',
 'diff': 'diff --git a/src/renderer/webgl/WebGLPipeline.js b/src/renderer/webgl/WebGLPipeline.js @@ -275,6 +275,10 @@ var WebGLPipeline = new Class({\noffset: offset\n});\n+ this.vertexComponentCount = Utils.getComponentCount(\n+ this.attributes,\n+ this.gl\n+ );\nreturn this;\n},\n',
 'lang': 'JavaScript',
 'chatgpt': 'Add vertexComponentCount property to WebGLPipeline'}

In [15]:
with open('../data/final_preprocessed_data/js_baseline/js_baseline_rag_300_prompt.json', 'r', encoding='UTF-8') as f:
  similar_diff = json.load(f)

In [18]:
lowest_item

{'msg': 'Fix user component assets',
 'diff': "diff --git a/src/components/User/User.js b/src/components/User/User.js @@ -9,8 +9,8 @@ const User = ({ user }) => {\nurl: user.avatar,\nalt: user.slug\n} : {\n- url: 'assets/avatar.png',\n- alt: 'no avatar'\n+ url: '/assets/avatar.png',\n+ alt: ''\n}\nreturn (\n",
 'lang': 'JavaScript',
 'chatgpt_rag': 'Update default avatar URL and alt text'}

In [19]:
zeroshot_data[lowest_index]

{'msg': 'Fix user component assets',
 'diff': "diff --git a/src/components/User/User.js b/src/components/User/User.js @@ -9,8 +9,8 @@ const User = ({ user }) => {\nurl: user.avatar,\nalt: user.slug\n} : {\n- url: 'assets/avatar.png',\n- alt: 'no avatar'\n+ url: '/assets/avatar.png',\n+ alt: ''\n}\nreturn (\n",
 'lang': 'JavaScript',
 'chatgpt': 'Fix avatar image path in User component'}

In [17]:
similar_diff[highest_index]

{'org_diff': 'diff --git a/src/renderer/webgl/WebGLPipeline.js b/src/renderer/webgl/WebGLPipeline.js @@ -275,6 +275,10 @@ var WebGLPipeline = new Class({\noffset: offset\n});\n+ this.vertexComponentCount = Utils.getComponentCount(\n+ this.attributes,\n+ this.gl\n+ );\nreturn this;\n},\n',
 'org_msg': 'update vertexComponentCount in WebGLPipeline',
 'sim_msg': 'remove left over debugging code',
 'sim_diff': 'diff --git a/src/video/webgl/webgl_compositor.js b/src/video/webgl/webgl_compositor.js @@ -500,7 +500,6 @@ class WebGLCompositor {\nclear(alpha = 0) {\nvar gl = this.gl;\ngl.clearColor(0, 0, 0, alpha);\n- console.log(alpha);\ngl.clear(gl.COLOR_BUFFER_BIT | gl.STENCIL_BUFFER_BIT);\n}\n'}

In [10]:
from tqdm import tqdm
with open('../data/final_preprocessed_data/js_baseline/js_baseline_rag_300.json', 'r', encoding='UTF-8') as f:
  data = json.load(f)

with open('../data/final_preprocessed_data/discriminator/dis_train_data.json', 'r', encoding='UTF-8') as f:
  test_data = json.load(f)[:100]

In [11]:
similar_item = []
for test_item in tqdm(test_data, total=len(test_data), desc="Processing documents"):
  max_score = 0
  max_item = None
  for db_item in db_data:
    cur_score = calculate_log_MNEXT_by_pair(db_item['msg'], test_item['msg'])
    if cur_score > max_score:
      max_item = db_item
      max_score = cur_score
  similar_item.append(max_item)

for sim_item, test_item in zip(similar_item, test_data):
  test_item['sim_diff'] = sim_item['diff']
  test_item['sim_msg'] = sim_item['msg']

with open('../data/final_preprocessed_data/discriminator/dis_train_data_top100_golden.json', 'w', encoding='UTF-8') as f:
  json.dump(test_data, f, ensure_ascii=False, indent=4)

Processing documents: 100%|██████████| 100/100 [1:25:37<00:00, 51.37s/it]


In [6]:
from tqdm.auto import tqdm

with open('../data/final_preprocessed_data/discriminator/dis_train_data_retrieve.json', 'r', encoding='UTF-8') as f:
  retrieve_data = json.load(f)

with open('../data/final_preprocessed_data/discriminator/dis_train_data_llm.json', 'r', encoding='UTF-8') as f:
  llm_data = json.load(f)
# with open('../data/final_preprocessed_data/js_baseline_test_data_300.json', 'r', encoding='UTF-8') as f:
#   test_data = json.load(f)

triplets = []
retrieve_count = 0
for llm_item, retrieve_item in tqdm(zip(llm_data, retrieve_data), total=len(llm_data), desc="Processing documents"):
    llm_score = calculate_log_MNEXT_by_pair(llm_item['race'], llm_item['msg'])
    retrieve_score = calculate_log_MNEXT_by_pair(retrieve_item['sim_msg'], retrieve_item['org_msg'])
    if retrieve_score > llm_score:
        label = 1
        retrieve_count+=1
    else:
        label = 0
    item = {'org_diff': retrieve_item['org_diff'], 'sim_diff': retrieve_item['sim_diff'], 'label': label}
    triplets.append(item)

Processing documents:   0%|          | 0/11884 [00:00<?, ?it/s]

ModuleNotFoundError: No module named 'langchain'

1299

In [8]:
with open('../data/final_preprocessed_data/discriminator/dis_train_data_with_label.json', 'w', encoding='UTF-8') as f:
    json.dump(triplets, f, ensure_ascii=False, indent=4)

In [7]:
# Extract diffs and assign IDs
train_samples = [{"id": i, "data": item["diff"]} for i, item in enumerate(test_data)]
database_samples = [{"id": i, "data": item["diff"]} for i, item in enumerate(db_data)]
similarity_scores = []
for i, similar_score in enumerate(similar_scores):
    similarity_scores.append({'train_id': i, 'scores': similar_score})

# Create the combined data structure
combined_data = {
    "train_samples": train_samples,
    "database_samples": database_samples,
    "similarity_scores": similarity_scores
}

# Save to JSON
with open('../data/final_preprocessed_data/code_retrieval/js_retrieve_test_data_4f.json', 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

In [12]:

for sim_item, test_item in zip(similar_item, test_data):
    if sim_item is not None:
        test_item['sim_diff'] = sim_item['diff'] if sim_item['diff'] is not None else ''
        test_item['sim_msg'] = sim_item['msg'] if sim_item['msg'] is not None else ''
    else:
        test_item['sim_diff'] = ''
        test_item['sim_msg'] = ''

with open('../data/final_preprocessed_data/js_baseline_test_data_300_golden.json', 'w', encoding='UTF-8') as f:
  json.dump(test_data, f, ensure_ascii=False, indent=4)

In [10]:
test_data[448]

{'msg': 'fix(babel-plugin-putout) strict-mode',
 'diff': 'diff --git a/packages/babel-plugin-putout/src/index.js b/packages/babel-plugin-putout/src/index.js @@ -31,6 +31,7 @@ export default function() {\n},\ngeneratorOverride(ast) {\n+ ast.program.directives = [];\nconst code = print(ast);\nreturn {code};\n},\n',
 'lang': 'JavaScript'}

In [9]:
for index, sim_item in enumerate(similar_item):
    if sim_item==None:
        print(index)

448


In [5]:
# lang = 'python'
for lang in ['python', 'js']:
    calculate_log_MNEXT(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_zeroshot.json', 'chatgpt')
    calculate_log_MNEXT(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag.json', 'chatgpt_rag')
    calculate_log_MNEXT(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_top4.json', 'chatgpt_rag')
    calculate_log_MNEXT(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_rag_sim_msg.json', 'chatgpt_rag')
    calculate_log_MNEXT(f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_cot.json', 'chatgpt_cot')

../data/final_preprocessed_data/python_baseline/python_baseline_zeroshot.json/chatgpt: 17.24000000000002
../data/final_preprocessed_data/python_baseline/python_baseline_rag.json/chatgpt_rag: 14.45300000000002
../data/final_preprocessed_data/python_baseline/python_baseline_rag_top4.json/chatgpt_rag: 14.475000000000012
../data/final_preprocessed_data/python_baseline/python_baseline_rag_sim_msg.json/chatgpt_rag: 16.65699999999999
../data/final_preprocessed_data/python_baseline/python_baseline_cot.json/chatgpt_cot: 16.99999999999998
../data/final_preprocessed_data/js_baseline/js_baseline_zeroshot.json/chatgpt: 16.04600000000001
../data/final_preprocessed_data/js_baseline/js_baseline_rag.json/chatgpt_rag: 15.019000000000013
../data/final_preprocessed_data/js_baseline/js_baseline_rag_top4.json/chatgpt_rag: 14.41900000000001
../data/final_preprocessed_data/js_baseline/js_baseline_rag_sim_msg.json/chatgpt_rag: 15.844000000000008
../data/final_preprocessed_data/js_baseline/js_baseline_cot.json/

In [9]:
with open('../data/final_preprocessed_data/python_baseline/python_baseline_rag.json', 'r', encoding='UTF-8') as f:
      org_data = json.load(f)

In [10]:
org_data[351]

{'msg': 'use shorthand method',
 'diff': 'diff --git a/examples/simulate_fight_scenario.py b/examples/simulate_fight_scenario.py @@ -36,7 +36,7 @@ class FightBot(BotAI):\n# destroy command center\nawait self._client.debug_kill_unit([cc.tag])\n# destroy all workers\n- for w in self.units(UnitTypeId.SCV):\n+ for w in self.workers:\nawait self._client.debug_kill_unit([w.tag])\n# create marines\nawait self._client.debug_create_unit([[UnitTypeId.MARINE, 4, p, me]])\n',
 'lang': 'Python',
 'chatgpt_rag': None}

In [12]:
calculate_log_MNEXT('../data/scenario/multi_diff/multi_diff_rag_react_v2.json', 'chatgpt_rag')
calculate_log_MNEXT('../data/scenario/multi_diff/multi_diff_rag_flask_v2.json', 'chatgpt_rag')
calculate_log_MNEXT('../data/scenario/multi_diff/multi_diff_rag_top4_flask_v2.json', 'chatgpt_rag')
calculate_log_MNEXT('../data/scenario/multi_diff/multi_diff_rag_top4_react_v2.json', 'chatgpt_rag')
calculate_log_MNEXT('../data/scenario/multi_diff/multi_diff_rag_sim_msg_flask_v2.json', 'chatgpt_rag')
calculate_log_MNEXT('../data/scenario/multi_diff/multi_diff_rag_sim_msg_react_v2.json', 'chatgpt_rag')

../data/scenario/multi_diff/multi_diff_rag_react_v2.json/chatgpt_rag: 9.889999999999997
../data/scenario/multi_diff/multi_diff_rag_flask_v2.json/chatgpt_rag: 13.779999999999998
../data/scenario/multi_diff/multi_diff_rag_top4_flask_v2.json/chatgpt_rag: 12.859999999999996
../data/scenario/multi_diff/multi_diff_rag_top4_react_v2.json/chatgpt_rag: 7.469999999999999
../data/scenario/multi_diff/multi_diff_rag_sim_msg_flask_v2.json/chatgpt_rag: 16.589999999999993
../data/scenario/multi_diff/multi_diff_rag_sim_msg_react_v2.json/chatgpt_rag: 8.379999999999997


In [6]:
file_path = '../data/chronicle/rag_dev/mxbai/chronicle_rag_dev_mxbai_query_sim_msg.json'
with open(file_path, 'r', encoding='UTF-8') as f:
  org_data = json.load(f)

In [6]:
calculate_log_MNEXT('chronicle_chatgpt_rag.json', 'chatgpt')

chatgpt: 15.79


In [7]:
calculate_log_MNEXT('chronicle_chatgpt_rag.json', 'chatgpt_rag')

chatgpt_rag: 6.519999999999998


In [8]:
calculate_log_MNEXT('chronicle_race_history.json', 'race_history')

race_history: 13.860000000000003
